In [4]:
!pip install scikit-image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 19.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.7 MB/s eta 0:00:00


In [6]:
pip install --force-reinstall scipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 827.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 42.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 28.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install --upgrade scipy


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 22.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 21.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.feature import hog,local_binary_pattern
import glob
import numpy as np
from sklearn import svm
from skimage import feature
from sklearn.metrics import classification_report
from skimage.color import rgb2gray

In [3]:
import sklearn


In [4]:
# import training data
live_train = glob.glob('Data/train_data/live/*.png') #positive
spoof_train = glob.glob('Data/train_spoof/live/*.png') # negative

# import testing data
live_test = glob.glob('Data/test_data/live/*.png') # positive
spoof_test = glob.glob('Data/test_spoof/live/*.png') # negative

In [5]:
live_test

[]

In [52]:
# initialize empty lists to append data
live_train_images = []
spoof_train_images = []
live_test_images = []
spoof_test_images = []

# read images from given path and append to the variavles
for path in live_train: live_train_images.append(imread(path))
for path in spoof_train: spoof_train_images.append(imread(path))
for path in live_test: live_test_images.append(imread(path))
for path in spoof_test: spoof_test_images.append(imread(path))

In [53]:
# convert images to np arrays
live_train_images, spoof_train_images = np.asarray(live_train_images), np.asarray(spoof_train_images)
live_test_images, spoof_test_images = np.asarray(live_test_images), np.asarray(spoof_test_images)

In [54]:
# change the type of feature you want to use here
feature_type = "lbp" #hog or lbp

# initialize empty lists to append features
live_train_features, spoof_train_features = [], []
live_test_features, spoof_test_features = [], []

if(feature_type == "hog"):
    
    # calculate and store hog features of training data
    for live_img, spoof_img in zip(live_train_images, spoof_train_images):
        live_train_features.append(hog(live_img, feature_vector = True))
        spoof_train_features.append(hog(spoof_img, feature_vector = True))
    
    # calculate and store hog features of testing data
    for live_img, spoof_img in zip(live_test_images, spoof_test_images):
        live_test_features.append(hog(live_img, feature_vector = True))
        spoof_test_features.append(hog(spoof_img, feature_vector = True))
        
if(feature_type == "lbp"):

# parameters copied from scikit docs of lbp
    METHOD = 'uniform'
    R = 3
    P = 8 * R

    # calculate and store lbp features of training data
    for live_img, spoof_img in zip(live_train_images, spoof_train_images):
        live_img = rgb2gray(live_img)
        spoof_img = rgb2gray(spoof_img)
        live_train_features.append(local_binary_pattern(live_img, P, R, METHOD).flatten())
        spoof_train_features.append(local_binary_pattern(spoof_img, P, R, METHOD).flatten())
    
    # calculate and store lbp features of training data   
    for live_img, spoof_img in zip(live_test_images, spoof_test_images):
        # convert rgb to gray
        live_img = rgb2gray(live_img)
        # convert rgb to gray
        spoof_img = rgb2gray(spoof_img)
        live_test_features.append(local_binary_pattern(live_img, P, R, METHOD).flatten())
        spoof_test_features.append(local_binary_pattern(spoof_img, P, R, METHOD).flatten())

In [55]:
# define labels 1 for live and 0 for spoof
# i.e. create a list of 200 1's and 200 0's 
labels_train = [[1]]*len(live_train_features) + [[0]]*len(spoof_train_features)

# merge both positive and negative training data
full_train = live_train_features + spoof_train_features

# assign labels for all test data, 1 for positive, 0 for negative
labels_test = [[1]]*len(live_test_features) + [[0]]*len(spoof_test_features)

# merge both positive and negative training data
full_test = live_test_features + spoof_test_features


In [56]:
len(labels_train)

400

In [57]:
clf = svm.SVC()
features_train = np.array(full_train)
features_test = np.array(full_test)

# stack features and labels together
data_frame_train = np.hstack((features_train,labels_train))

# shuffle train data
np.random.shuffle(data_frame_train)

In [58]:
# split features and labels of train data
x_train = data_frame_train[:len(data_frame_train),:-1]
y_train = data_frame_train[:len(data_frame_train),-1]

# ready test data and labels
x_test = features_test
y_test = labels_test

In [59]:
# fit the model with train set
clf.fit(x_train,y_train)

SVC()

In [60]:
# predict labels: y_pred from x_test
y_pred = clf.predict(x_test)

In [48]:
# calculate other metrics
print(classification_report(y_test, y_pred))

# hog: (0.859375, 0.845, 0.8434343434343435, None)
# lbp: (0.9303430011968119, 0.9225, 0.9221452744065148, None)

              precision    recall  f1-score   support

           0       0.79      0.94      0.86       200
           1       0.93      0.74      0.83       200

    accuracy                           0.84       400
   macro avg       0.86      0.84      0.84       400
weighted avg       0.86      0.84      0.84       400



In [61]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.99      0.93       200
           1       0.99      0.85      0.92       200

    accuracy                           0.92       400
   macro avg       0.93      0.92      0.92       400
weighted avg       0.93      0.92      0.92       400

